# Introducción a mongodb

la instalación de mongodb está en la máquina virtual del curso en

    /opt/mongo
    
y los comandos están en el path.

estamos usando mongo 3.0: https://docs.mongodb.com/v3.0/

este tutorial se enfoca en el **USO** de mongodb. no en su **configuración** o **despliegue** en un cluster

limpia el directorio de la base de datos y arranca el demonio

    rm -rf /data/db/*
    monngod

conéctate con el CLI

    mongo

comandos (dbs y colecciones son creados de manera implícita y no tienen que tener una estructura predeterminada):

    show dbs
    use mydb
    db.casas.insert({ "id": 209384, "ciudad": "roma", "superficie": 120})
    db.casas.insert({ "id": 209387, "propietaro": "giuseppe", "habitaciones": 3})
    db.casas.insert({ "id": 209398, "hipoteca": "no",  "ciudad": "roma"})    
    db.casas.insert({ "id": 209389, "ciudad": "roma"})    
    db.casas.insert({ "id": 209289, "ciudad": "madrid", "superficie": 89})
    db.casas.insert({ "id": 209182, "ciudad": "torino", "superficie": 109})
    
    db.casas.find({"ciudad": "roma"})
    db.casas.count( {"superficie": {$gt: 100}})    
    
    for (i=0; i<30; i++) { db.casas.insert({ "id": 209182, "ciudad": "torino", "superficie": i*10+10}) }    

    db.casas.find()
    it
    
    cursor = db.casas.find( {"superficie": {$gt: 100}})
    cursor.next()
    cursor.explain()
    
    
    db.casas.drop()
    
revisa la doc de operaciones CRUD:  https://docs.mongodb.com/v3.0/core/crud-introduction/

revisa operaciones agregadas: https://docs.mongodb.com/manual/aggregation/


# `pymongo`

see https://api.mongodb.com/python/current/tutorial.html

In [1]:
import pymongo as pm

In [4]:
client = MongoClient('mongodb://localhost:27017/')

In [5]:
db = client.mydb

In [16]:
c = db.casas.find()

In [17]:
for i in c:
    print i

{u'superficie': 120.0, u'ciudad': u'roma', u'_id': ObjectId('59e4de056e9928de92b98a90'), u'id': 209384.0}
{u'propietaro': u'giuseppe', u'_id': ObjectId('59e4de056e9928de92b98a91'), u'habitaciones': 3.0, u'id': 209387.0}
{u'ciudad': u'roma', u'_id': ObjectId('59e4de056e9928de92b98a92'), u'id': 209398.0, u'hipoteca': u'no'}
{u'ciudad': u'roma', u'_id': ObjectId('59e4de056e9928de92b98a93'), u'id': 209389.0}
{u'superficie': 89.0, u'ciudad': u'madrid', u'_id': ObjectId('59e4de056e9928de92b98a94'), u'id': 209289.0}
{u'superficie': 109.0, u'ciudad': u'torino', u'_id': ObjectId('59e4de056e9928de92b98a95'), u'id': 209182.0}
{u'superficie': 10.0, u'ciudad': u'torino', u'_id': ObjectId('59e4de056e9928de92b98a96'), u'id': 209182.0}
{u'superficie': 20.0, u'ciudad': u'torino', u'_id': ObjectId('59e4de056e9928de92b98a97'), u'id': 209182.0}
{u'superficie': 30.0, u'ciudad': u'torino', u'_id': ObjectId('59e4de056e9928de92b98a98'), u'id': 209182.0}
{u'superficie': 40.0, u'ciudad': u'torino', u'_id': Obje

## map-reduce

In [18]:
from bson.code import Code
mapper = Code("""
               function () {
                   emit(this.ciudad, this.superficie);
               }
               """)

In [35]:
 reducer = Code("""
                function (key, values) {
                  var total = 0;
                  for (var i = 0; i < values.length; i++) {
                    total += values[i];
                  }
                  return total;
                }
                """)

In [36]:
result = db.casas.map_reduce(mapper, reducer, "rr")

In [37]:
for doc in result.find():
    print doc

{u'_id': None, u'value': None}
{u'_id': u'madrid', u'value': 89.0}
{u'_id': u'roma', u'value': 120.0}
{u'_id': u'torino', u'value': 4759.0}


### Testing javascript

use node.js command line through ipython magics to execute any javascript

In [62]:
%%script node

// a dictionary
var a = {
  FirstName: "Chris",
  "one": 1,
  1: "some value"
};

console.log("hola", a[1], "adios")
console.log("name:", a.FirstName)

// splitting text
text = "testing from  js"

var res = text.split(" ")
console.log("text is: ", res)

for (i=0; i<res.length; i++) {
    console.log(i,res[i], res[i].length)
}

hola some value adios
name: Chris
text is:  [ 'testing', 'from', '', 'js' ]
0 'testing' 7
1 'from' 4
2 '' 0
3 'js' 2
